What are the available fields?

There's no "description" or "plot summary" of movies.
So one must look at all the dialogs and decipher the movie genre?

How can we do this?
Let's do this in a naive way first, then let's try to get creative.

---

Destruction of the data:
- movies
    - conversations
        - dialogs
            - sentences
                - words

---

## Naive way:

- Average all word embeddings in a dialog to get dialog embeddings.
- Average all dialog embeddings to get movie plot embedding.

Thought process:
  - We get a sentence embedding by averaging the word embeddings
  - In the same way, can we get "scene" embedding by averaging all the sentence embeddings in a scene?
  - Then averaging all scene embeddings should give us movie embedding.
  - We don't have scenes, do we? If not, we can skip scenes, will averaging all dialog embeddings give us movie embedding?

Problems:

- The conversations might mostly include "everyday lines" that do not relate to the plot.
- For example, "You must learn how to lie", "they have to!" do not say anything about the plot.

## Another method:

- Find all named entities (of some relevant category).
- Then get the dialogs with named entities only.
- Assumption: dialogs with named entities will contain plot-related information.
-

---

## Rule-based method for baseline.

For each genre, what are the most used words? (after removing the stopwords)
How many of these words occur for multiple movies?

Intuitive rationale:

- [Within one genre] a genre-defining word will likely have high frequency in a movie and among all movies of the same genre.
- [Among genres] a genre-defining word will likely have high frequency in movies within the genre and low frequency among movies of other genres.

This can be obtained by finding the tf-idf value for each word in the corpus.

If frequency of a word is low among movies of the same genre, then it is likely not genre-defining.

Assuming that we obtained these "genre-defining" words for each genre,
the next task would be to somehow learn that these words (or similar words) will help predict the movie's genre.

### Implementation:

1. Parse all the data.
2. store all the genres.
3. Split 95:5 of the movies randomly (less data problem!)
4. Find tf-idf of each word in the training set. document = genre.
5. Can these be custom named entities?

Part 1:

Well, we need lines for each genre. We don't care about the "sequential" nature of dialogs. Just need all of them.

Specifically, we need a structure like this:

- genres
    - movies
        - words

Here words are obtained by:

- spaCy tokenize each dialog.
- spaCy lemmatize
- spaCy remove stopwords

Note that a movie can have multiple genres. So the dialogs for two genres might repeat.

Part 2:


---

Extracting all information about a movie.


In [241]:
import re
from collections import Counter, OrderedDict
from contextlib import contextmanager
from copy import deepcopy
from os import listdir, makedirs
from os.path import join
from typing import List, Tuple, Dict, Optional, TextIO

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
import spacy
from pydantic import BaseModel
from tqdm import tqdm
import heapq

In [2]:

class Movie(BaseModel):
    id: str
    title: str
    release_year: int
    imdb_rating: float
    num_votes: int
    genres: List[str] = []
    characters: dict = {}
    conversations: list = []


class Character(BaseModel):
    name: str
    id: str
    gender: Optional[str]  # m|f|None
    credit_position: Optional[int]
    movie: Movie

    def __eq__(self, other):
        return self.id == other.id


class Dialog(BaseModel):
    id: str
    speaker: Character
    listener: Optional[Character]
    dialog: str


class Conversation(BaseModel):
    characters: Tuple[Character, Character]
    dialogs: List[Dialog]

In [3]:
def update_movie_characters(movies: Dict[str, Movie], characters_path: str):
    with open(characters_path, "r", encoding="ISO-8859-1") as characters_file:
        for character in characters_file:
            # sample: u0 +++$+++ BIANCA +++$+++ m0 +++$+++ 10 things i hate about you +++$+++ f +++$+++ 4
            character = character.strip().split(" +++$+++ ")
            character_id, name, movie_id, movie_title, gender, position = character

            gender = None if gender == "?" else gender
            position = None if position == "?" else position

            assert movie_id in movies
            movie = movies[movie_id]

            character = Character(
                name=name,
                id=character_id,
                gender=gender,  # m|f|None
                credit_position=position,
                movie=movie,
            )

            movie.characters[character_id] = character


In [4]:

def get_all_dialogs(movies: Dict[str, Movie], lines_path: str) -> Dict[str, Dialog]:
    all_dialogs = {}
    with open(lines_path, "r", encoding="ISO-8859-1") as dialogs_file:
        for dialog in dialogs_file:
            # Sample: L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!
            # Note: Sometimes, there's no dialog, so splitting with ' +++$+++ ' will be wrong (with space at the end.)
            #       So, we will split without spaces on either side and then strip
            dialog = dialog.strip().split("+++$+++")
            dialog = [d.strip() for d in dialog]
            dialog_id, character_id, movie_id, character_name, dialog = dialog

            assert movie_id in movies
            movie = movies[movie_id]

            assert character_id in movie.characters
            speaker = movie.characters[character_id]

            dialog = Dialog(id=dialog_id, speaker=speaker, dialog=dialog)
            all_dialogs[dialog_id] = dialog
    return all_dialogs


In [273]:

def update_movie_dialogs(
    movies: Dict[str, Movie], conversations_path: str, lines_path: str
):
    all_dialogs: Dict[str, Dialog] = get_all_dialogs(movies, lines_path)

    with open(conversations_path, "r", encoding="ISO-8859-1") as conversations_file:
        for conversation in conversations_file:
            # Sample: u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']
            conversation = conversation.strip().split(" +++$+++ ")
            character1, character2, movie_id, dialogs = conversation

            dialogs = dialogs.strip("][").replace("'", "").split(", ")

            assert movie_id in movies
            movie = movies[movie_id]

            assert character1 in movie.characters
            assert character2 in movie.characters

            character1 = movie.characters[character1]
            character2 = movie.characters[character2]

            for dialog in dialogs:
                assert dialog in all_dialogs
                dialog = all_dialogs[dialog]
                listener = character1 if character2 == dialog.speaker else character2
                dialog.listener = listener

            conversation = Conversation(
                characters=(character1, character2),
                dialogs=[all_dialogs[dialog] for dialog in dialogs],
            )

            movie.conversations.append(conversation)


In [274]:

def _preprocess_year(year: str) -> int:
    if "/I" in year:
        year = year.replace("/I", "")
    year = int(year)
    return year


def get_movies_dict(movie_titles_metadata_path: str) -> Dict[str, Movie]:
    movies_dict = {}
    with open(movie_titles_metadata_path, "r", encoding="ISO-8859-1") as movies_file:
        for movie in movies_file:
            movie = movie.strip().split(" +++$+++ ")
            movie_id, title, year, rating, votes, genres = movie
            # Replace list representation with list of strings
            genres = genres.strip("][").replace("'", "").split(", ")
            year = _preprocess_year(year)
            rating = float(rating)

            movie = Movie(
                id=movie_id,
                title=title,
                release_year=year,
                imdb_rating=rating,
                num_votes=votes,
                genres=genres,
            )

            movies_dict[movie_id] = movie

    return movies_dict


In [275]:

def get_data(path: str):
    files = listdir(path)

    # 1. For each movie:
    #    a. get all characters
    #    b. get all conversations

    assert "movie_titles_metadata.txt" in files
    assert "movie_characters_metadata.txt" in files

    # This will likely use more memory.
    # movies_df = pd.read_table(
    #     join(path, 'movie_titles_metadata.txt'),
    #     sep=r' \+\+\+\$\+\+\+ ',
    #     encoding='ISO-8859-1',
    #     header=None,
    # )

    movies_dict = get_movies_dict(join(path, "movie_titles_metadata.txt"))
    update_movie_characters(movies_dict, join(path, "movie_characters_metadata.txt"))
    update_movie_dialogs(
        movies_dict,
        join(path, "movie_conversations.txt"),
        join(path, "movie_lines.txt"),
    )

    return movies_dict


data_path = "/Users/akhil/code/lexical_lab/companies/ginger/data/cornell"
movies = get_data(data_path)

617it [00:00, 14818.83it/s]
83097it [00:06, 12755.38it/s]


In [8]:
list(movies.keys())[:5]

['m0', 'm1', 'm2', 'm3', 'm4']

In [9]:
m0 = movies['m0']

In [10]:
list(m0.characters.keys())[:5]

['u0', 'u1', 'u2', 'u3', 'u4']

In [11]:
conversations = m0.conversations[:5]
conversations[0].dialogs[0].dialog

'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.'

In [251]:
all_genres = [genre for _, movie in movies.items() for genre in movie.genres]
genre_movies_count = Counter(all_genres)
all_genres = list(set(all_genres))

# Sorting by value to see which genres have the most movies.
sorted(genre_movies_count.items(), key=lambda item: item[1], reverse=True)

[('drama', 320),
 ('thriller', 269),
 ('action', 168),
 ('comedy', 162),
 ('crime', 147),
 ('romance', 132),
 ('sci-fi', 120),
 ('adventure', 116),
 ('mystery', 102),
 ('horror', 99),
 ('fantasy', 78),
 ('short', 5),
 ('animation', 2),
 ('family', 1)]

In [252]:
# Well, there's one movie without any genre. So let's put that as 'other'
all_genres.append('other')
all_genres.remove('')

genre_movies_count['other'] = genre_movies_count['']
del genre_movies_count['']

# Let's also replace '' with 'other' in all movies
for _, movie in movies.items():
    if '' in movie.genres:
        movie.genres.append('other')
        movie.genres.remove('')

ValueError: list.remove(x): x not in list

In [253]:
# Some of the genres have very low movie count.
# This will make the frequency based method very difficult.
# We might employ other techniques for these low-frequent genres.
# For now, to make the problem easier, let's only use the high frequent genres only.

# Eliminating all genres with frequency < 75 (75 is just based on intuition.)
low_frequency_genres = [genre for genre in all_genres if genre_movies_count[genre] <= 75]

for genre in low_frequency_genres:
    # Remove from all_genres
    all_genres.remove(genre)

    # Remove from movie.genres
    for _, movie in movies.items():
        if genre in movie.genres:
            movie.genres.remove(genre)

# Remove movies with empty genres
movies_to_remove = []
for movie_id, movie in movies.items():
    if len(movie.genres) == 0:
        movies_to_remove.append(movie_id)
for movie_id in movies_to_remove:
    del movies[movie_id]

In [254]:
# [NOT USED] Creates text files with all dialogs of each genre. NOT USED, but works.

@contextmanager
def get_genre_dialogs_files(output_path: str, genres: List[str], mode: str):
    """
    A wrapper that encapsulates handling multiple genre-dialogs files.
    """
    makedirs(output_path, exist_ok=True)
    files: Dict[str, TextIO] = {genre: open(join(output_path, f'{genre}.txt'), mode) for genre in genres}

    try:
        yield files
    finally:
        for genre in genres:
            files[genre].close()


def create_genre_dialogs_files():
    # Creating dialogs text files for each genre.
    genre_dialogs_path = '/Users/akhil/code/lexical_lab/companies/ginger/data/output/genre_dialogs'
    with get_genre_dialogs_files(genre_dialogs_path, all_genres, 'w') as genre_dialogs_files:
        for _, movie in movies.items():
            for conversation in movie.conversations:
                for dialog in conversation.dialogs:
                    for genre in movie.genres:
                        genre_dialogs_files[genre].write(dialog.dialog + '\n')


In [255]:
"""
Creating a structure like this:
- genres
    - movies
        - words
"""

spacy_nlp = spacy.load('en_core_web_sm', disable=['tok2vec', 'parser', 'ner'])

def extract_words(text):
    # Without this, there will a ' ' (space) word
    text = re.sub(r'\s+', ' ', text)
    doc = spacy_nlp(text)
    # Words without stopwords
    words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return words

extract_words('This is an example of  a sentence. Akhil\'s friends have many cars')

['example', 'sentence', 'akhil', 'friends', 'cars']

In [256]:
# Find word counts for each genre.
genre_words_count = {genre: {} for genre in all_genres}

for _, movie in tqdm(movies.items()):
    dialogs = '. '.join([d.dialog for conversation in movie.conversations for d in conversation.dialogs])
    words = extract_words(dialogs)
    word_frequencies = Counter(words)

    for genre in movie.genres:
        for word, count in word_frequencies.items():
            genre_word_count = genre_words_count[genre].get(word, {'movie_count': 0, 'count': 0})
            genre_word_count['movie_count'] += 1
            genre_word_count['count'] += count
            genre_words_count[genre][word] = genre_word_count

100%|██████████| 611/611 [01:11<00:00,  8.58it/s]


In [263]:
# Remove words that are too infrequent.

# Anything count < infrequent_threshold is removed.
infrequent_threshold = 3

# At least this % of movies should contain the genre-defining words
infrequent_movie_threshold = 0.25

for genre, words_count in genre_words_count.items():
    words_to_remove = []
    for word, word_count in words_count.items():
        word_frequency = word_count['count']  # / word_count['movie_count']
        movie_frequency = word_count['movie_count'] / genre_movies_count[genre]
        if word_frequency < infrequent_threshold or movie_frequency < infrequent_movie_threshold:
            words_to_remove.append(word)
            # print(f'{word=}, {word_count=}')
    for word in words_to_remove:
        del words_count[word]

In [264]:
# Building word frequency table.

# all words in this frequency table
all_words = list(set([word for word_counts in genre_words_count.values() for word in word_counts]))
len(all_words)

1015

In [265]:
frequency_matrix = np.zeros((len(all_genres), len(all_words)), dtype=int)

for i, genre in enumerate(all_genres):
    for j, word in enumerate(all_words):
        if word in genre_words_count[genre]:
            frequency = genre_words_count[genre][word]
            frequency = frequency['count']  # / frequency['movie_count']
            frequency_matrix[i][j] = frequency

frequency_matrix

array([[516, 259,   0, ...,   0, 222, 154],
       [273, 137,   0, ...,   0, 130,  72],
       [121,  51,   0, ...,   0,  73,  39],
       ...,
       [461, 132,   0, ...,   0, 131, 108],
       [210, 104,   0, ...,   0,  89,  75],
       [336, 154, 133, ...,   0, 131, 108]])

In [266]:
inter_genre_word_counts = frequency_matrix.sum(0)
inter_genre_word_counts

array([3540, 1486,  133, ...,   52, 1390,  846])

In [267]:
tfidf = frequency_matrix / inter_genre_word_counts
tfidf

array([[0.14576271, 0.17429341, 0.        , ..., 0.        , 0.15971223,
        0.1820331 ],
       [0.07711864, 0.09219381, 0.        , ..., 0.        , 0.09352518,
        0.08510638],
       [0.03418079, 0.03432032, 0.        , ..., 0.        , 0.05251799,
        0.04609929],
       ...,
       [0.13022599, 0.08882907, 0.        , ..., 0.        , 0.0942446 ,
        0.12765957],
       [0.05932203, 0.06998654, 0.        , ..., 0.        , 0.06402878,
        0.08865248],
       [0.09491525, 0.10363392, 1.        , ..., 0.        , 0.0942446 ,
        0.12765957]])

In [270]:
# Okay, now that we have tf-idf matrix,
# Let's find the highest ranking top k words for each genre.

for i in range(len(all_genres)):
    tfidf_ranks = tfidf[i]
    counts = frequency_matrix[i]

    zipped = zip(tfidf_ranks, all_words, counts, inter_genre_word_counts)
    pairs = heapq.nlargest(10, zipped)
    keywords = [pair[1] for pair in pairs]
    print(f'{all_genres[i]}: {keywords}')

thriller: ['tight', 'killer', 'loose', 'department', 'address', 'split', 'fuckin', 'checked', 'key', 'cops']
action: ['south', 'weapon', 'guard', 'sooner', 'computer', 'guns', 'weapons', 'learned', 'cross', 'orders']
fantasy: ['trick', 'tree', 'purpose', 'opened', 'magic', 'hoping', 'feed', 'evil', 'matters', 'greatest']
drama: ['writing', 'runs', 'accept', 'lawyer', 'agree', 'sleeping', 'decide', 'won', 'plans', 'laugh']
romance: ['wind', 'willing', 'wild', 'train', 'sold', 'sing', 'round', 'ring', 'recognize', 'raise']
adventure: ['sea', 'river', 'quickly', 'operation', 'north', 'lord', 'jump', 'enemy', 'continue', 'code']
sci-fi: ['systems', 'study', 'science', 'noticed', 'nature', 'level', 'form', 'exist', 'emergency', 'data']
horror: ['monster', 'bodies', 'locked', 'evil', 'dreams', 'danger', 'asleep', 'lock', 'radio', 'box']
comedy: ['whoa', 'um', 'u', 'song', 'screw', 'perfectly', 'pants', 'pal', 'older', 'natural']
mystery: ['rules', 'proof', 'murdered', 'investigation', 'held'

'this is a test'